# Taller 1

### librerias

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### importar dataset

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')

### seleccion de variables

In [3]:
X = dataset.iloc[:,3:13].values
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [4]:
y = dataset.iloc[:,13].values
y = np.array(y,dtype=np.float32)
y

array([1., 0., 1., ..., 1., 1., 0.], dtype=float32)

### codificacion datos categoricos

In [5]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [6]:
labelencoder_X_1=LabelEncoder()
X[:,1]=labelencoder_X_1.fit_transform(X[:,1])
X

array([[619, 0, 'Female', ..., 1, 1, 101348.88],
       [608, 2, 'Female', ..., 0, 1, 112542.58],
       [502, 0, 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 0, 'Female', ..., 0, 1, 42085.58],
       [772, 1, 'Male', ..., 1, 0, 92888.52],
       [792, 0, 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [7]:
labelencoder_X_2=LabelEncoder()
X[:,2]=labelencoder_X_2.fit_transform(X[:,2])
X

array([[619, 0, 0, ..., 1, 1, 101348.88],
       [608, 2, 0, ..., 0, 1, 112542.58],
       [502, 0, 0, ..., 1, 0, 113931.57],
       ...,
       [709, 0, 0, ..., 0, 1, 42085.58],
       [772, 1, 1, ..., 1, 0, 92888.52],
       [792, 0, 0, ..., 1, 0, 38190.78]], dtype=object)

#### creacion de las variables dummies

cuando se cambia una variable categorica en una nueva clase 

In [8]:
transformer = ColumnTransformer(
    transformers=[
        ("OneHot",        # Just a name
         OneHotEncoder(), # The transformer class
         [1]              # The column(s) to be applied on.
         )
    ],
    remainder='passthrough' # donot apply anything to the remaining columns
)
X = transformer.fit_transform(X.tolist())
X = np.array(X,dtype=np.float32)
X.dtype

dtype('float32')

### Division de los datos de prueb a y entrenamiento

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

### Ajustar las escalas

In [10]:
from sklearn.preprocessing import StandardScaler

In [11]:
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

# Creacion de la Red Neuronal

### importacion de las librerias

In [12]:
import keras
from keras.models import Sequential #arquitectura
from keras.layers import Dense      #capa de la red  

2022-03-27 13:48:08.397711: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


### inicialziacion de la red neuronal

In [25]:
red = Sequential()

### capa de entrada

tenemos 11 variables de entrada y una de salida (si,no)
ademas de construiran 2 capas ocultas de 12 neuronas cada una


In [26]:
capa_1=Dense(24,input_dim=10,use_bias=1,activation='relu')
capa_2=Dense(24,use_bias=1,activation='relu')
capa_3=Dense(1,use_bias=1,activation='sigmoid')

In [27]:
red.add(capa_1)
red.add(capa_2)
red.add(capa_3)

### activacion

In [28]:
red.compile(
    optimizer='adam',
    loss='binary_crossentropy', 
    metrics=['accuracy']
)

In [29]:
np.shape(X_train)

(8000, 10)

### entrenamiento

In [30]:
%%timeit 
red.fit(X_train,
        y_train,
        epochs=10,
        batch_size=20
       )

Epoch 1/10
400/400 [==============================] - 0s 576us/step - loss: 0.5184 - accuracy: 0.7746
Epoch 2/10
400/400 [==============================] - 0s 472us/step - loss: 0.4048 - accuracy: 0.8305
Epoch 3/10
400/400 [==============================] - 0s 436us/step - loss: 0.3700 - accuracy: 0.8486
Epoch 4/10
400/400 [==============================] - 0s 715us/step - loss: 0.3671 - accuracy: 0.8457
Epoch 5/10
400/400 [==============================] - 0s 501us/step - loss: 0.3467 - accuracy: 0.8549
Epoch 6/10
400/400 [==============================] - 0s 463us/step - loss: 0.3454 - accuracy: 0.8566
Epoch 7/10
400/400 [==============================] - 0s 436us/step - loss: 0.3426 - accuracy: 0.8613
Epoch 8/10
400/400 [==============================] - 0s 472us/step - loss: 0.3338 - accuracy: 0.8603
Epoch 9/10
400/400 [==============================] - 0s 455us/step - loss: 0.3316 - accuracy: 0.8650
Epoch 10/10
400/400 [==============================] - 0s 485us/step - loss: 0.326

### predicción

In [31]:
y_pred=red.predict(X_test)
y_pred

array([[0.2510985 ],
       [0.19416219],
       [0.17340502],
       ...,
       [0.2646865 ],
       [0.2585357 ],
       [0.22510865]], dtype=float32)

In [32]:
y_pred = np.round(y_pred)
y_pred

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

### comparacion de resultados

In [33]:
from sklearn.metrics import confusion_matrix
cm= confusion_matrix(y_test,y_pred)

In [34]:
cm

array([[1471,  124],
       [ 178,  227]])

array([[1506,   89],
       [ 189,  216]], dtype=int64)

In [35]:
scores = red.evaluate(X_train, y_train)
print("\n%s: %.2f%%" % (red.metrics_names[1], scores[1]*100))

250/250 [==============================] - 0s 351us/step - loss: 0.2960 - accuracy: 0.8785

accuracy: 87.85%


In [36]:
scores = red.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (red.metrics_names[1], scores[1]*100))

63/63 [==============================] - 0s 684us/step - loss: 0.3529 - accuracy: 0.8490

accuracy: 84.90%


In [37]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

         0.0       0.92      0.89      0.91      1649
         1.0       0.56      0.65      0.60       351

    accuracy                           0.85      2000
   macro avg       0.74      0.77      0.75      2000
weighted avg       0.86      0.85      0.85      2000

